# **Inference pipeline**
___  
  
In this notebook we show how to apply **inference** with [BiaPy](https://biapyx.github.io/).  

**Without any coding**, we explain step by step how to
1. **upload a set of test images** with their corresponding labels,
2. **apply the model** to the test images, and
4. **download the results** to your local machine.

**Disclaimer:** the structure of the notebook is heavily inspired in the fantastic [ZeroCostDL4Mic notebooks](https://github.com/HenriquesLab/ZeroCostDL4Mic/wiki).

**Contact:** This notebook was created by [Ignacio Arganda-Carreras](mailto:ignacio.arganda@ehu.eus), [Lenka Backová](mailto:lenka.backova@ehu.eus), [Daniel Franco-Barranco](mailto:daniel.franco@dipc.org) and [Ane Paniagua](mailto:anepaniagua@gmail.com). For suggestions, comments, or issues, please reach out to us via email or [create an issue in BiaPy's repository](https://github.com/BiaPyX/BiaPy/issues). Thank you!


## **Expected inputs and outputs**
___
**Inputs**

This notebook expects three folders as input:
* **Test raw images**: with the raw images to test the model.
* **Test labels**: ground truth of the test images. Depending on the workflow to be used, the inputs will be images or CSV files.
* **Output folder**: a path to store the results.

**Outputs**

Depending on the workflow, the output could be an image, or a csv file per each test sample.

<font color='red'><b>Note</b></font>: for testing purposes, you can also run this notebook with the **example datasets provided in 'Manage file(s) source > Option 3'**.

**Data structure**

To ensure the proper operation of the library the data directory tree should be something like this:

```
dataset/
└── test
    ├── raw
    │   ├── testing-0001.tif
    │   ├── testing-0002.tif
    │   ├── . . .
    │   └── testing-9999.tif
    └── gt
        ├── testing_groundtruth-0001.tif
        ├── testing_groundtruth-0002.tif
        ├── . . .
        └── testing_groundtruth-9999.tif
```

**⚠️ Warning:** Ensure that images and their corresponding ground truth files are sorted in the same way. A common approach is to fill with zeros the image number added to the filenames (as in the example).

**Input Format Support**

This notebook is compatible with a range of input formats. You can use the following file extensions: `.tif`, `.png`, `.jpg`, `.npy`, `.h5`, `.hdf5` (every extension supported by [scikit-image](https://scikit-image.org/docs/stable/api/skimage.io.html#skimage.io.imread)).





## **Prepare the environment**
___

Establish connection with Google services. You **must be logged in to Google** to continue.
Since this is not Google's own code, you will probably see a message warning you of the dangers of running unfamiliar code. This is completely normal.



## **Check for GPU access**
---

By default, the session should be using Python 3 and GPU acceleration, but it is possible to ensure that these are set properly by doing the following:

Go to **Runtime -> Change the Runtime type**

**Runtime type: Python 3** *(Python 3 is programming language in which this program is written)*

**Accelerator: GPU** *(Graphics processing unit)*

## **Install BiaPy**
---

In [ ]:
#@markdown ##Play to reinstall Colab libraries so they are compatible wiht BiaPy

#@markdown ⚠️ **Wait until this cell is fully executed to run the next one** ⚠️
#@markdown This will make the notebook restart. Do not worry and continue executing the next cell once this one is finished.

# uninstall Colab's numpy
!pip uninstall -y numpy --quiet

# install BiaPy's compatible libraries
!pip install scipy==1.13.* numpy==1.26.* zarr==2.18.* --quiet

# Restart the runtime
print('Stopping runtime...')
exit()
print('You can run the next cell now.')

#%load_ext autoreload
#%autoreload 2
#import numpy as np
#import scipy
#print(scipy.__version__)
#print(np.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 890.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 31.7 MB/s eta 0:00:00
Stopping runtime...
You can run the next cell now.


In [ ]:
#@markdown ##Play to install BiaPy and its dependences

!pip install biapy==3.5.12

# Then install Pytorch + CUDA 11.8
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu118

# Finally install some packages that rely on the Pytorch installation
!pip install timm pytorch-msssim torchmetrics[image]==1.4.*


import os
import sys
import numpy as np
from tqdm.notebook import tqdm
from skimage.io import imread
import ipywidgets as widgets
from ipywidgets import Output
from biapy import BiaPy

changed_source = False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/

## **Manage file(s) source**
---
The input folder can be provided using three different options: by directly uploading the folder (option 1), by using a folder stored in Google Drive (option 2) or by using a few samples of our data (option 3).

Depending on the option chosen, different steps will have to be taken, as explained in the following cells.


### **Option 1: Upload Files from Your Local Machine**
---
You will be prompted to upload your files to Colab and they will be stored under `/content/input/`.

In [ ]:
#@markdown ##Play the cell to upload local files (test raw images)

from google.colab import files
import os
input_dir = '/content/input/test/raw'

if os.path.exists(input_dir):
    # Ask the user if they want to delete the existing items in the folder
    delete_items = ''
    while not delete_items in ['y', 'n']:
        delete_items = input("Do you want to delete the existing items in the folder? (yes[y]/no[n]): ").strip().lower()
    if delete_items == 'y':
        for delete_root, delete_dirs, delete_files in os.walk(input_dir, topdown=False):
            for name in delete_files:
                os.unlink(os.path.join(delete_root, name))
else:
    # Ensure the directory exists
    os.makedirs(input_dir, exist_ok=True)


%cd {input_dir}
uploaded = files.upload()
%cd /content

In [ ]:
#@markdown ##Play the cell to upload local files (test ground truth)

from google.colab import files
import os
input_dir = '/content/input/test/gt'

if os.path.exists(input_dir):
    # Ask the user if they want to delete the existing items in the folder
    delete_items = ''
    while not delete_items in ['y', 'n']:
        delete_items = input("Do you want to delete the existing items in the folder? (yes[y]/no[n]): ").strip().lower()
    if delete_items == 'y':
        for delete_root, delete_dirs, delete_files in os.walk(input_dir, topdown=False):
            for name in delete_files:
                os.unlink(os.path.join(delete_root, name))
else:
    # Ensure the directory exists
    os.makedirs(input_dir, exist_ok=True)


%cd {input_dir}
uploaded = files.upload()
%cd /content

In [ ]:
# @title  { display-mode: "form" }
#@markdown ##Play the cell to upload local files (YAML file)

from google.colab import files
%cd /content
uploaded = files.upload()

In [ ]:
# @title  { display-mode: "form" }
#@markdown ##Play the cell to upload local files (model's weights)

from google.colab import files
%cd /content
uploaded = files.upload()


### **Option 2: Mount Your Google Drive**
---
To use this notebook on your own data from Google Drive, you need to mount Google Drive first.

Play the cell below to mount your Google Drive and follow the link that will be shown. In the new browser window, select your drive and select 'Allow', copy the code, paste into the cell and press enter. This will give Colab access to the data on the drive.

Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab

#@markdown * Click on the URL.

#@markdown * Sign in your Google Account.

#@markdown * Copy the authorization code.

#@markdown * Enter the authorization code.

#@markdown * Click on "Files" site on the right. Refresh the site. Your Google Drive folder should now be available here as "drive".

# mount user's Google Drive to Google Colab.
from google.colab import drive
drive.mount('/content/gdrive')

## **Paths for Input Images and Output Files**
___

Depending on the option you chose for managing file sources, you'll set your paths differently:

- **Option 1 (Upload from Local Machine)**:
  - Set `test_data_path` to `/content/input/test/raw`
  - Set `test_data_gt_path` to `/content/input/test/gt`
  - Set `yaml_file` to `/content/your_yaml_file.yaml`
  - Set `checkpoint_file` to `/content/your_checkpoint.pth`
  - Set `output_path` to `/content/out`
  
- **Option 2 (Use Google Drive Data)**:
  - Insert the paths to your input files and your desired output directory here, i.e., `/content/gdrive/MyDrive/...`.
  
**Note**: Ensure you download your results from the `/content/out` directory after the process!

**Helpful Tip**: If you're unsure about the paths to your folders, look at the top left of this notebook for a small folder icon. Navigate through the directories until you locate your desired folder. Right-click on it and select "Copy Path" to copy the folder's path.

In [ ]:
#@markdown #####Path to test images
test_data_path = '/content/input/test/raw' #@param {type:"string"}
#@markdown #####Path to test ground truth (if exists)
test_data_gt_path = '/content/input/test/gt' #@param {type:"string"}
#@markdown #####Path to the YAML configuration file
yaml_file = '/content/my_2d_semantic_segmentation.yaml' #@param {type:"string"}
#@markdown #####Path to checkpoint file
checkpoint_file = '/content/my_2d_semantic_segmentation_1-checkpoint-best.pth' #@param {type:"string"}
#@markdown #####Path to store the resulting images (it'll be created if not existing):
output_path = '/content/output' #@param {type:"string"}

## **Configure and apply your DNN model**
---

### **Select your parameters**
---
* **`load_gt`:** Select to load ground truth labels and measure output performance. **Default value: True**

In [ ]:
load_gt = True #@param {type:"boolean"}

### **Make the inference**
---

In [ ]:
#@markdown ##Play to run the model
import os
import errno

os.chdir('/content/')

# Edit previous configuration file if it exists to load the checkpoint model
if os.path.exists( yaml_file ):
    import yaml
    with open( yaml_file, 'r') as stream:
        try:
            biapy_config = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    biapy_config['PATHS'] = {}
    biapy_config['PATHS']['CHECKPOINT_FILE'] = checkpoint_file
    biapy_config['MODEL'] = {}
    biapy_config['MODEL']['LOAD_CHECKPOINT'] = True

    # save file
    with open( yaml_file, 'w') as outfile:
        yaml.dump(biapy_config, outfile, default_flow_style=False)

# Check folders before modifying the .yaml file
if not os.path.exists(test_data_path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), test_data_path)
ids = sorted(next(os.walk(test_data_path))[2])
if len(ids) == 0:
    raise ValueError("No images found in dir {}".format(test_data_path))

if not os.path.exists(yaml_file):
    raise ValueError("No YAML configuration file found in {}".format(yaml_file))

if not os.path.exists(checkpoint_file):
    raise ValueError("No h5 checkpoint file found in {}".format(checkpoint_file))


# open template configuration file
import yaml
with open( yaml_file, 'r') as stream:
    try:
        biapy_config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

biapy_config['DATA']['TEST']['PATH'] = test_data_path
biapy_config['DATA']['TEST']['GT_PATH'] = test_data_gt_path

biapy_config['DATA']['TEST']['LOAD_GT'] = load_gt
biapy_config['TRAIN']['ENABLE'] = False
biapy_config['TEST']['ENABLE'] = True
biapy_config['MODEL']['LOAD_CHECKPOINT']= True
biapy_config['PATHS'] = {}
biapy_config['PATHS']['CHECKPOINT_FILE']= checkpoint_file

# save file
with open( yaml_file, 'w') as outfile:
    yaml.dump(biapy_config, outfile, default_flow_style=False)

print( "Inference configuration finished.")

job_name = os.path.splitext(yaml_file)[0].split('/')[-1]

# Run the code
biapy = BiaPy(yaml_file, result_dir=output_path, name=job_name, run_id=1, gpu=0)
biapy.run_job()


[11:45:38.918311] Inference configuration finished.
[11:45:38.929211] Date: 2024-06-18 11:45:38
[11:45:38.929341] Arguments: Namespace(config='/content/my_2d_semantic_segmentation.yaml', result_dir='/content/output', name='my_2d_semantic_segmentation', run_id=1, gpu=0, world_size=1, local_rank=-1, dist_on_itp=False, dist_url='env://', dist_backend='nccl')
[11:45:38.929389] Job: my_2d_semantic_segmentation_1
[11:45:38.929438] Python       : 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
[11:45:38.929491] PyTorch:  2.2.0+cu118
[11:45:38.930854] Not using distributed mode
[11:45:38.938360] Configuration details:
[11:45:38.938444] AUGMENTOR:
  AFFINE_MODE: reflect
  AUG_NUM_SAMPLES: 10
  AUG_SAMPLES: True
  BRIGHTNESS: False
  BRIGHTNESS_EM: False
  BRIGHTNESS_EM_FACTOR: (-0.1, 0.1)
  BRIGHTNESS_EM_MODE: 3D
  BRIGHTNESS_FACTOR: (-0.1, 0.1)
  BRIGHTNESS_MODE: 3D
  CBLUR_DOWN_RANGE: (2, 8)
  CBLUR_INSIDE: True
  CBLUR_SIZE: (0.2, 0.4)
  CHANNEL_SHUFFLE: False
  CMIX_SIZE: (0.2, 0.4)
  CN

100%|██████████| 1/1 [00:00<00:00, 690.99it/s]


[11:45:38.983584] *** Loaded data shape is (1, 768, 1024, 1)
[11:45:38.983699] 3) Loading test masks . . .
[11:45:38.983743] Loading data from /content/input/test/y


100%|██████████| 1/1 [00:00<00:00, 790.78it/s]

[11:45:38.991918] *** Loaded data shape is (1, 768, 1024, 1)
[11:45:38.992099] ###############
[11:45:38.992145] # Build model #
[11:45:38.992176] ###############


...

[11:45:39.289100] Processing image: testing-0158.tif
[11:45:42.851767] Saving (1, 768, 1024, 1) data as .tif in folder: /content/output/my_2d_semantic_segmentation/results/my_2d_semantic_segmentation_1/full_image



100%|██████████| 1/1 [00:00<00:00,  6.27it/s]
                                             

[11:45:43.049115] Saving (1, 768, 1024, 1) data as .tif in folder: /content/output/my_2d_semantic_segmentation/results/my_2d_semantic_segmentation_1/full_image_binarized



100%|██████████| 1/1 [00:03<00:00,  3.79s/it]

[11:45:43.069417] Releasing memory . . .
[11:45:43.069520] #############
[11:45:43.069576] #  RESULTS  #
[11:45:43.069610] #############
[11:45:43.069687] Loss (per image): 1.2073482275009155
[11:45:43.069734] Test Foreground IoU (per image): 0.05902527784686678
[11:45:43.069770] Test Overall IoU (per image): 0.4835477549131152
[11:45:43.069803]  
[11:45:43.069878] FINISHED JOB my_2d_semantic_segmentation_1 !!


## **Download results**
---

In [ ]:
#@markdown ###Play to download a zip file with all the results in test.

from google.colab import files

os.chdir('/content/')

!zip -r -q /content/output.zip $output_path

files.download("/content/output.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Acknowledgments**
---
We would like to acknowledge the inspiration provided by the excellent [ZeroCostDL4Mic notebooks](https://github.com/HenriquesLab/ZeroCostDL4Mic/wiki).